#Assignment completed
Members
- Sai Teja Eleti
- Saikiran pothukani **bold text**

**In this tutorial we learned how to use Apache spark and large-scale data(agents.json) processing like removing commas, converting to lower case and spliting, Understood the spark RDD's And explored the sparksql and dataframes with given condition.**

# Spark (with PySpark)


In this tutorial, you will learn how to use Apache Spark, a framework for large-scale data processing, within a notebook.
Upon completing this lab you will be able to :
 + Program in Spark with the Python Language
 + Demonstrate how to read and process data using Spark
 + Compare and contrast RDD and Dataframes.
 + Build a simple machine learning application with Spark.



# Requirements : (Recommended) Just complile it in Google Colab


# Else: Installing Spark, PySpark and configuration to run it on jupyter notebooks

According to your OS, you have to follow the following tutorial :

+ [Windows](https://changhsinlee.com/install-pyspark-windows-jupyter/)
+ [Mac Os X](https://jmedium.com/pyspark-in-mac/)

At the end of the installation, you should be able to run the following code that is a kind of Hello word in PySpark.

In [1]:
!pip install findspark

In [2]:
!pip install pyspark

In [3]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.sql('''select 'spark' as hello ''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [16]:
spark.stop()

## Part 1 :  Spark Context

When writing a spark program, the first thing to do is to define a `SparkContext`.

In Spark, communication occurs between a driver and executors. The driver has Spark jobs that it needs to run and these jobs are split into tasks that are submitted to the executors for completion. The results from these tasks are delivered back to the driver.

Here, we will use the `findspark` package that has to be installed using the following command:

` pip3 install findspark`

Then we can use the `findspark.init()`function to locate the Spark process.




In [44]:

import re
import os
from pyspark.sql import functions as fn

In [14]:
# Find and initialize spark.
findspark.init()

In [17]:
spark.stop()

In [19]:
try:
    sc
except NameError:
    # Initialize Spark Context.
    sc = pyspark.SparkContext("local[*]", "First program in Spark")

In [30]:
# Download and extract the text file (if needed)
data_dir = "/content/SparkData/"
file_name = "pg345.txt"
file_path = os.path.join(data_dir, file_name)
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Created directory: {data_dir}")


Try printing out sc to see its type.

In [27]:
# Load the data.
data = sc.textFile(file_path)
print(type(data))

<class 'pyspark.rdd.RDD'>


### WordCount in Spark
In this part, we will write the wordcount in Spark and apply it on the novel Dracula of Bram Stocker (from the Gutemberg project).

First read the [pg345.txt](./SparkData/pg345.txt) file.

To see the content, of the file, we need to run the action `collect` on the  RDD `data`

In [28]:
data.collect()

['The Project Gutenberg eBook of Dracula, by Bram Stoker',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org. If you are not located in the United States, you',
 'will have to check the laws of the country where you are located before',
 'using this eBook.',
 '',
 'Title: Dracula',
 '',
 'Author: Bram Stoker',
 '',
 'Release Date: October, 1995 [eBook #345]',
 '[Most recently updated: November 12, 2023]',
 '',
 'Language: English',
 '',
 'Produced by: Chuck Greif and the Online Distributed Proofreading Team',
 '',
 '*** START OF THE PROJECT GUTENBERG EBOOK DRACULA ***',
 '',
 '',
 '',
 '',
 '                                DRACULA',
 '',
 '                                  _by_',
 '',
 '                            

## Part 2 : Understanding Spark RDD’s

With the RDD `data`, from the previous cell, execute a `flatMap()` for each line in the input and then convert it to lower case, remove the commas, split the words on a space and store in the RDD `words`

In [31]:
words = data.flatMap(lambda line: re.sub(r'[^a-zA-Z\s]', '', line.lower()).split())

Execute the command with the action `take()` and retrieve the first 10 words from the `flatMap()`transformation

In [34]:
print("First 10 words:", words.take(10))

First 10 words: ['the', 'project', 'gutenberg', 'ebook', 'of', 'dracula', 'by', 'bram', 'stoker', 'this']


Perform a classic `map()` to create a tuple where each word has a count of 1

In [35]:
word_pairs = words.map(lambda word: (word, 1))

Write the reducing function

In [36]:
word_counts = word_pairs.reduceByKey(lambda a, b: a + b)


Get all words that occur more than once and sort them alphabetically:

In [37]:
filtered_words = word_counts.filter(lambda x: x[1] > 1).sortByKey()


Now, get all words that occur more than once and sort them by frequency. Select the first 20. Hints : look at the `takeOrdered` action.

In [38]:
top_words = word_counts.sortBy(lambda x: x[1], ascending=False).take(20)

### A simple exercice

+ Create a Python collection of 10,000 integers
+ Create a Spark base RDD from that collection
+ Subtract one from each value using map
+ Perform action collect to view results
+ Perform action count to view counts
+ Apply transformation filter and view results with collect

In [39]:
nums = sc.parallelize(range(10000))  # Create RDD of 10,000 integers
nums_minus_one = nums.map(lambda x: x - 1)  # Subtract one from each value
print("First 10 values after subtraction:", nums_minus_one.take(10))
print("Total count:", nums_minus_one.count())

First 10 values after subtraction: [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
Total count: 10000


### An improved WordCount

Print the top 10 most frequent words with their probability of appearance


In [40]:
total_word_count = word_counts.map(lambda x: x[1]).sum()
word_probabilities = word_counts.map(lambda x: (x[0], x[1] / total_word_count))

In [41]:
print("Top 10 words with probability:", word_probabilities.sortBy(lambda x: -x[1]).take(10))

Top 10 words with probability: [('the', 0.04925758632930606), ('and', 0.03607408223470799), ('i', 0.028821318840047495), ('to', 0.027793078966374108), ('of', 0.02291506004186405), ('a', 0.01810436634717785), ('in', 0.015662296647203554), ('he', 0.015564369040187041), ('that', 0.015019646726157687), ('it', 0.013091696963020087)]


Get rid of special characters (.,:!?')


Identify the transformations and the actions in your script


How many times are the transformations evaluated? (Hint: it depends)
In our case the transformation evaluated 53 times

In [42]:
word_counts.persist()

PythonRDD[35] at RDD at PythonRDD.scala:53

Can you reduce this number? (Hint: check out "persist")

In [ ]:
# TO DO

In [43]:
sc.stop()

## Part 3 : Spark SQL and dataframes

In this part, you will explore Spark DataFrames and the SQL Context. In particular, we will work on a database that contains a sample of the world population by working on data that comes from [pplapi](http://pplapi.com/). The file [agents.json](./SparkData/agents.json) is a file that was extracted from this api using the following command :


`wget https://s3-eu-west-1.amazonaws.com/course.oc-static.com/courses/4297166/agents.json`

In [48]:
# Download the agents.json file
os.system("wget https://s3-eu-west-1.amazonaws.com/course.oc-static.com/courses/4297166/agents.json")

0

In [49]:
# Initialize Spark Session
spark = SparkSession.builder.appName("SparkSQL_DataFrames").getOrCreate()

In [50]:
# Load JSON file into DataFrame
df = spark.read.json("agents.json")

Print the 5 first lines of the dataframe `df`

In [51]:
df.show(5)

+------------+----------+------------------+------------------+------+
|country_name|        id|          latitude|         longitude|   sex|
+------------+----------+------------------+------------------+------+
|       China| 227417393| 33.15219798270325|100.85840672174572|  Male|
|       Haiti|6821129477|19.325567983697297|-72.43795260265814|Female|
|       India|2078667700|23.645271492037235| 80.85636526088884|Female|
|       China| 477556555| 33.45864668881662| 93.33604038078953|Female|
|       India|1379059984|28.816938290678692|  80.7728698035823|Female|
+------------+----------+------------------+------------------+------+
only showing top 5 rows



Write the spark instructions that enable to display the number of French agents.

In [52]:
french_agents_count = df.filter(df.country_name == "France").count()
print(f"Number of French agents: {french_agents_count}")

Number of French agents: 94


Write the spark instructions that enable to display the number of Indian female agents.

In [54]:
indian_female_agents_count = df.filter((df.country_name == "India") & (df.sex == "Female")).count()
print(f"Number of Indian female agents: {indian_female_agents_count}")

Number of Indian female agents: 828


Using the notion of temporary view (function `createTempView`), create a temporary view associated to the dataframe `df`.

In [58]:
df.createOrReplaceTempView("agents_table")


Write some SQL query on the resulting table as shown in the example below.

In [59]:
spark.sql("SELECT country_name,id FROM agents_table ORDER BY id DESC LIMIT 10").show()

+-----------------+----------+
|     country_name|        id|
+-----------------+----------+
| French Polynesia|7170821229|
|       Cabo Verde|7167692449|
|         Suriname|7166451460|
|         Suriname|7166235088|
|            Macau|7166034642|
|       Montenegro|7164357515|
|Equatorial Guinea|7163867872|
|           Bhutan|7163256789|
|           Bhutan|7163004645|
|           Bhutan|7162877973|
+-----------------+----------+



In [62]:
spark.stop()
sc.stop()

You can also create a dataframe from an existing RDD as shown on the example below

In [63]:
sc=pyspark.SparkContext("local[*]","Dataframe examples in Spark")
sqlContext = pyspark.SQLContext(sc)



/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [64]:
documents_rdd = sc.parallelize([
        [1, 'cats are cute', 0],
        [2, 'dogs are playfull', 0],
        [3, 'lions are big', 1],
        [4, 'cars are fast', 1]])
users_rdd = sc.parallelize([
        [0, 'Alice', 20],
        [1, 'Bob', 23],
        [2, 'Charles', 32]])

In [65]:
documents_df = documents_rdd.toDF(['doc_id', 'text', 'user_id'])
users_df = users_rdd.toDF(['user_id', 'name', 'age'])

# printing the inferred schema for documents
documents_df.printSchema()


root
 |-- doc_id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: long (nullable = true)



Some functions can be apply to a column or to different columns. Here, we compute the age avarage in the dataframe `users_df`.

In [66]:
from pyspark.sql import functions as fn

user_age_df = users_df.select(fn.avg('age')).show()

+--------+
|avg(age)|
+--------+
|    25.0|
+--------+




Compute the max of age in the dataframe `users_df`.

Join (as in SQL) the two dataframes `users_df` and `documents_df`

In [69]:
inner_join_df = users_df.join(documents_df, on='user_id', how='inner')
inner_join_df.show()


+-------+-----+---+------+-----------------+
|user_id| name|age|doc_id|             text|
+-------+-----+---+------+-----------------+
|      0|Alice| 20|     1|    cats are cute|
|      0|Alice| 20|     2|dogs are playfull|
|      1|  Bob| 23|     3|    lions are big|
|      1|  Bob| 23|     4|    cars are fast|
+-------+-----+---+------+-----------------+



Outer Join (Left) (as in SQL) the two dataframes `users_df` and `documents_df`

In [70]:
left_outer_join_df = users_df.join(documents_df, on='user_id', how='left')
left_outer_join_df.show()

+-------+-------+---+------+-----------------+
|user_id|   name|age|doc_id|             text|
+-------+-------+---+------+-----------------+
|      0|  Alice| 20|     2|dogs are playfull|
|      0|  Alice| 20|     1|    cats are cute|
|      1|    Bob| 23|     4|    cars are fast|
|      1|    Bob| 23|     3|    lions are big|
|      2|Charles| 32|  NULL|             NULL|
+-------+-------+---+------+-----------------+

